In [1]:
import sxs
import pandas as pd
import numpy as np
import os

catalog = sxs.load("catalog")

In [2]:
ROOT_PATH = current_folder = globals()['_dh'][0]
SLURM_PATH = os.path.join(ROOT_PATH, 'slurm_scripts/')

In [3]:
CCE = True

if CCE:
    cat_name = 'CCE'
else:
    cat_name = 'SXS'

In [4]:
if CCE:
    SXS_num_screened = ['0001', '0002', '0003', '0004', '0005', '0006', '0007',  
                        '0010', '0011', '0012']
else:
    sxs_df = catalog.simulations_dataframe
    relevant_sims = sxs_df[(sxs_df["object_types"] == "BHBH") & ((sxs_df["reference_eccentricity"] < 0.001) | (np.isnan(sxs_df["reference_eccentricity"]))) &
        (sxs_df["reference_chi1_perp"] < 0.001) & (sxs_df["reference_chi2_perp"] < 0.001)]
    sxs_compt = sxs_df[["reference_mass_ratio", 'reference_dimensionless_spin1_mag', 'reference_dimensionless_spin2_mag',
                        "reference_eccentricity", "reference_chi1_perp", "reference_chi2_perp"]]
    pd.set_option("display.max_rows", 10)
    SXS_num_list = [str[8:] for str in relevant_sims.index.to_list()]
    # screen
    SXS_num_screened = []

    for SXS_num in SXS_num_list:
        if "0209" <= SXS_num <= "0305" or "1419" <= SXS_num <= "1509":
            SXS_num_screened.append(SXS_num)
        #     pass
        # else:
        #     SXS_num_screened.append(SXS_num)

In [5]:
runname = "normal2_prodB1"
setting_name = "normal2"
flags = "-l"
if CCE:
    flags += " -cce"

os.makedirs(os.path.join(SLURM_PATH, f"slurm_out/{cat_name}_fit_{runname}"), exist_ok=True)

In [6]:
SXS_list_path = os.path.join(SLURM_PATH, f"{cat_name}_list_{runname}.txt")

with open(SXS_list_path, "w") as f:
    for SXS_num in SXS_num_screened:
        f.write(f"{SXS_num}\n")

In [13]:
slurm_script = f"""#!/bin/bash
#SBATCH --job-name="{cat_name}_fit_{runname}"
#SBATCH --output="./slurm_out/{cat_name}_fit_{runname}/{cat_name}_fit_{runname}.%A.%a.out"
#SBATCH --partition=shared
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=4Gb
#SBATCH --account=cit122
#SBATCH -t 48:00:00
#SBATCH --mail-type=ALL
#SBATCH --mail-user=hcheung5@jhu.edu
#SBATCH --array=0-{len(SXS_num_screened)-1}

pwd; hostname; date

source /home/mcheung1/Ringdown/jaxqualin/bin/activate
export PYTHONPATH=$PYTHONPATH:/home/mcheung1/Ringdown/jaxqualin/Jaxqualin

mapfile -t {cat_name}_nums < {SXS_list_path}

python /home/mcheung1/Ringdown/jaxqualin/Jaxqualin/SXS_fit.py"""\
+ r" ${" + f"{cat_name}" + r"_nums[$SLURM_ARRAY_TASK_ID]} "\
+ f"""{setting_name} {runname} {flags}

date"""

In [14]:
slurm_script_path = os.path.join(SLURM_PATH, f"{cat_name}_fit_{runname}.sbatch")

with open(slurm_script_path, "w") as f:
    f.write(slurm_script)